https://www.analyticsvidhya.com/blog/2020/09/alternative-hyperparameter-optimization-technique-you-need-to-know-hyperopt/

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

# import warnings
# warnings.filterwarnings("ignore")

In [2]:
data = Path(os.getenv('DATA')) / 'wholesale_customers_data/Wholesale customers data.csv'


In [3]:
df = pd.read_csv(data)

In [4]:
y = df.pop('Channel')
X = df
# y = df['Channel']

In [5]:
X.head()

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,3,12669,9656,7561,214,2674,1338
1,3,7057,9810,9568,1762,3293,1776
2,3,6353,8808,7684,2405,3516,7844
3,3,13265,1196,4221,6404,507,1788
4,3,22615,5410,7198,3915,1777,5185


In [6]:
y.head()

0    2
1    2
2    2
3    1
4    2
Name: Channel, dtype: int64

In [7]:
y[y == 2] = 0

y[y == 1] = 1

In [8]:
y.head()


0    0
1    0
2    0
3    1
4    0
Name: Channel, dtype: int64

In [9]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [10]:
# X_scaled

In [12]:
space = {
    "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
}

In [13]:
# define objective function

def hyperparameter_tuning(params):
    clf = RandomForestClassifier(**params,n_jobs=-1)
    acc = cross_val_score(clf, X_train, y_train, scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [ ]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

 75%|██████████████████████████████████▌           | 75/100 [02:04<00:50,  2.03s/trial, best loss: -0.9318878900052882]

In [ ]:
trials.results


In [ ]:
trials.losses()


In [ ]:
best

In [ ]:
cleaned_hyperparams = {param:int(val) if float(val).is_integer() else val for param, val in best.items()}

In [ ]:
cleaned_hyperparams

In [ ]:
model = xgb.XGBClassifier(**cleaned_hyperparams)

In [ ]:
evaluation = [( X_train, y_train), ( X_test, y_test)]

In [ ]:
model.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10, verbose=True)

In [ ]:
preds = model.predict(X_test)

In [ ]:
accuracy_score(y_test, preds)